In [1]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

In [2]:
end = date.today()
print(f'Year = {end.year}; month= {end.month}; day={end.day}')

start = date(year=end.year-70, month=end.month, day=end.day)
print(f'Period for indexes: {start} to {end} ')

Year = 2024; month= 4; day=21
Period for indexes: 1954-04-21 to 2024-04-21 


# Q1

In [3]:
gdpc1 = pdr.DataReader("GDPC1", "fred", start=start)

In [4]:
gdpc1.iloc[:8]

,GDPC1
DATE,
1954-07-01,2880.482
1954-10-01,2936.852
1955-01-01,3020.746
1955-04-01,3069.910
1955-07-01,3111.379
1955-10-01,3130.068
1956-01-01,3117.922
1956-04-01,3143.694


In [5]:
# Formula: https://www.wallstreetprep.com/knowledge/year-over-year-yoy/
gdpc1['gdpc1_yoy'] = (gdpc1.GDPC1/gdpc1.GDPC1.shift(4))-1

In [6]:
gdpc1.iloc[-4:]

,GDPC1,gdpc1_yoy
DATE,,
2023-01-01,22112.329,0.017179
2023-04-01,22225.350,0.023825
2023-07-01,22490.692,0.029269
2023-10-01,22679.255,0.031345


In [7]:
print(f"{100*gdpc1.iloc[-4:].gdpc1_yoy.mean():.1f}")

2.5


# Q2

In [8]:
DGS2 = pdr.DataReader("DGS2", "fred", start=start)
DGS10 = pdr.DataReader("DGS10", "fred", start=start)

In [9]:
DGS2.head()

,DGS2
DATE,
1976-06-01,7.26
1976-06-02,7.23
1976-06-03,7.22
1976-06-04,7.12
1976-06-07,7.09


In [10]:
DGS10.head()

,DGS10
DATE,
1962-01-02,4.06
1962-01-03,4.03
1962-01-04,3.99
1962-01-05,4.02
1962-01-08,4.03


In [11]:
joined = DGS10.join(DGS2, how="inner", on="DATE")

In [12]:
joined["dgs10-dgs2"] = joined["DGS10"] - joined["DGS2"]
joined.head()

,DGS10,DGS2,dgs10-dgs2
DATE,,,
1976-06-01,7.94,7.26,0.68
1976-06-02,7.94,7.23,0.71
1976-06-03,7.92,7.22,0.70
1976-06-04,7.89,7.12,0.77
1976-06-07,7.88,7.09,0.79


In [13]:
joined_after_2000 = joined[joined.index >= "2000-01-01"]

In [14]:
print(f"{joined_after_2000['dgs10-dgs2'].min():.1f}")

-1.1


# Q3

In [15]:
import yfinance as yf

In [16]:
sp500 = yf.download(tickers="^GSPC", 
                    period = "max",
                    interval = "1d")
ipc_m = yf.download("^MXX",
                    period = "max",
                    interval = "1d")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [17]:
def calculate_growth(index: pd.DataFrame, start_time: str, end_time: str, state: str) -> int:
    return int((index[index.index==end_time][state].values[0]/index[index.index==start_time][state].values[0] - 1)*100)

In [18]:
calculate_growth(sp500, "2019-04-09", "2024-04-09", "Close"), calculate_growth(sp500, "2019-04-09", "2024-04-09", "Open"), calculate_growth(sp500, "2019-04-09", "2024-04-09", "Adj Close")

(81, 80, 81)

In [19]:
calculate_growth(ipc_m, "2019-04-09", "2024-04-09", "Close"), calculate_growth(ipc_m, "2019-04-09", "2024-04-09", "Open"), calculate_growth(ipc_m, "2019-04-09", "2024-04-09", "Adj Close")

(27, 27, 27)

# Q4

In [20]:
import requests

In [21]:
response = requests.get("https://companiesmarketcap.com/most-profitable-companies/?download=csv", headers={
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
})
data = response.content.decode("utf-8").split('\n')

top6 = [row.split(",")[2].replace("\"", "") for row in data[1:7]]
top6

['2222.SR', 'BRK-B', 'AAPL', 'MSFT', 'GOOG', 'JPM']

In [22]:
for index_name in top6:
    index = yf.download(tickers=index_name, 
                        period = "max",
                        interval = "1d")
    index_2023 = index[(index.index >= "2023-01-01") & (index.index < "2024-01-01")]
    index_2023_max = index_2023["Adj Close"].max()
    index_2023_min = index_2023["Adj Close"].min()
    
    print(f"{index_name} = {(index_2023_max-index_2023_min)/index_2023_max:.2f}")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


2222.SR = 0.21
BRK-B = 0.21


[*********************100%%**********************]  1 of 1 completed


AAPL = 0.37


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


MSFT = 0.42
GOOG = 0.39


[*********************100%%**********************]  1 of 1 completed

JPM = 0.28


# Q5

In [23]:
tickers = yf.Tickers(" ".join(n.lower() for n in top6))

In [24]:
for index_name in top6:
    dividends = tickers.tickers[index_name].dividends
    dividends_2023 = sum(dividends[(dividends.index >= "2023-01-01") & (dividends.index < "2024-01-01")])

    index = yf.download(tickers=index_name, 
                        period = "max",
                        interval = "1d")
    index_2023 = index[(index.index >= "2023-01-01") & (index.index < "2024-01-01")]
    
    print(f"{index_name} = {100*dividends_2023/index_2023.iloc[-1]['Adj Close']:.1f}%")

[*********************100%%**********************]  1 of 1 completed


2222.SR = 2.8%


[*********************100%%**********************]  1 of 1 completed


BRK-B = 0.0%


[*********************100%%**********************]  1 of 1 completed


...

MSFT = 0.7%


[*********************100%%**********************]  1 of 1 completed


GOOG = 0.0%


[*********************100%%**********************]  1 of 1 completed

JPM = 2.4%
